# Get Distances to POIs

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import logging

import sys

sys.path.append("/app")

# import scraping as sc

import pandas as pd

from jinja2 import Template

from db_utils import get_engine, get_table_creation_query

from gis_utils import get_gdf_coords, get_closest_idxs, get_closest_pois_slow


import dataloader as loader


import numpy as np
import os

import geopandas as gpd
from network_utils import get_closest_osmids


<IPython.core.display.Javascript object>

## Connect to Database

In [3]:
user = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")
host = os.getenv("POSTGRES_HOST")

<IPython.core.display.Javascript object>

In [4]:
engine = get_engine(user, password, host)

<IPython.core.display.Javascript object>

## Define search parameters

In [5]:
searchname = "tamzin"

<IPython.core.display.Javascript object>

## Load data

In [6]:
q_addresses = f"""SELECT c.* FROM 
{searchname}.bng_coords c
INNER JOIN
{searchname}.address_ids_to_process a
ON c.address_id=a.address_id"""

<IPython.core.display.Javascript object>

In [7]:
with engine.connect() as conn:
    df_nodes = pd.read_sql("SELECT * FROM node_coords", con=conn)

<IPython.core.display.Javascript object>

In [8]:
df_addresses = loader.load_sql(q_addresses, user=user, password=password, host=host)

<IPython.core.display.Javascript object>

## Snap to network

In [9]:
coords_nodes = df_nodes[["eastings", "northings"]].values

<IPython.core.display.Javascript object>

In [10]:
closest_osmids = get_closest_osmids(
    df_addresses[["eastings", "northings"]].values,
    coords_nodes,
    df_nodes.osmid.values,
)

<IPython.core.display.Javascript object>

In [11]:
output = pd.DataFrame(
    {
        "address_id": df_addresses.address_id,
        "osmid": closest_osmids,
    }
)

<IPython.core.display.Javascript object>

In [12]:
cols = {
    "address_id": "INTEGER",
    "osmid": "BIGINT",
}

index_cols = ["address_id", "osmid"]
unique_cols = ["address_id"]

<IPython.core.display.Javascript object>

In [13]:
create_q = get_table_creation_query(
    "address_nodes", cols, searchname, index_cols, unique_cols
)

<IPython.core.display.Javascript object>

In [14]:
with engine.connect() as conn:
    conn.execute(create_q)

<IPython.core.display.Javascript object>

In [16]:
with engine.connect() as conn:
    output.to_sql(
        "address_nodes", schema=searchname, index=False, con=conn, if_exists="append"
    )

<IPython.core.display.Javascript object>